In [1]:

import numpy as np
import pygmo as pg

In [2]:
class test_rust:
    def __init__(self, dim):
        self.dim = dim
    def fitness(self, x):
        A = 10
        return [A * x.size + np.sum(x ** 2 - A * np.cos(2*np.pi*x))]
    def get_bounds(self):
        return ([-5.12] * self.dim, [5.12] * self.dim)

    def get_name(self):
        return "Функция Растригина"

    def get_extra_info(self):
        return "\tРазмерность: " + str(self.dim)

In [3]:
class test_StyblinskiTang:
    def __init__(self, dim):
        self.dim = dim
    def fitness(self, x):
        return [np.sum(x ** 4 - 16 * x ** 2 + 5 * x)/2]
    def get_bounds(self):
        return ([-5] * self.dim, [5] * self.dim)

    def get_name(self):
        return "Функция Стыбинского-Танга"

    def get_extra_info(self):
        return "\tРазмерность: " + str(self.dim)

In [4]:
problem_rust = pg.problem(test_rust(dim=2))
print(problem_rust)

Problem name: Функция Растригина
	C++ class name: class pybind11::object

	Global dimension:			2
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [-5.12, -5.12]
	Upper bounds: [5.12, 5.12]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: none

Extra info:
	Размерность: 2


In [5]:
problem_ST = pg.problem(test_StyblinskiTang(dim=2))
print(problem_ST)

Problem name: Функция Стыбинского-Танга
	C++ class name: class pybind11::object

	Global dimension:			2
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [-5, -5]
	Upper bounds: [5, 5]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: none

Extra info:
	Размерность: 2


In [6]:
alg_list = [pg.de(gen=100), pg.pso(gen=100), pg.sade(gen=100)]
problems = [problem_rust, problem_ST]
print(alg_list)

[<pygmo.core.de object at 0x0000023CB469E0F0>, <pygmo.core.pso object at 0x0000023CB469F4B0>, <pygmo.core.sade object at 0x0000023CB46C62F0>]


# Эволюция
## Differential Evolution: 
Основан на идеях эволюционных стратегий. Использует популяцию векторов параметров и проводит операции мутации, кроссовера и селекции для поиска оптимальных параметров. Мутации производятся путем комбинирования различных индивидов в популяции.

## Particle Swarm Optimization: 
Моделирует поведение роя частиц в пространстве параметров. Каждая частица движется по пространству, обновляя свое положение в зависимости от своей лучшей позиции и лучшей позиции в рое. Таким образом, алгоритм старается найти оптимальное положение в пространстве параметров.

## Self-Adaptive Differential Evolution:
Расширение Differential Evolution с механизмом автоматической настройки параметров мутации. Позволяет алгоритму самостоятельно адаптироваться к характеру задачи оптимизации в процессе выполнения.

In [7]:
info = []
for alg in alg_list:
    algorithm_rust = pg.algorithm(alg)
    population_rust = pg.population(problem_rust, size=15)
    bsr = population_rust.champion_x
    bvr = population_rust.champion_f

    bsr_best = bsr
    bvr_best = bvr

    bsr_worst = bsr
    bvr_worst = bvr

# population_rust = algorithm.evolve(population_rust)

    mse_max = 0
    mse_min = np.sum((bsr - np.zeros_like(bsr)) ** 2) / bsr.size
    T = 25
    mse_av = 0.0
    for _ in range(T):
        population_rust = algorithm_rust.evolve(population_rust)
        bsr = population_rust.champion_x
        bvr = population_rust.champion_f
        mse = np.sum((bsr - np.zeros_like(bsr)) ** 2) / bsr.size
        if (mse < mse_min):
            mse_min = mse
            bsr_best = bsr
            bvr_best = bvr
        elif (mse > mse_max):
            mse_max = mse
            bsr_worst = bsr
            bvr_worst = bvr



        mse_av += mse

    mse_av /=T
    info.append([mse_av, mse_min, mse_max, bvr_best[0], bvr_worst[0]])
  

In [8]:
for alg in alg_list:
    algorithm_rust = pg.algorithm(alg)
    population_rust = pg.population(problem_ST, size=15)
    bsr = population_rust.champion_x
    bvr = population_rust.champion_f

    bsr_best = bsr
    bvr_best = bvr

    bsr_worst = bsr
    bvr_worst = bvr

# population_rust = algorithm.evolve(population_rust)

    mse_max = 0
    mse_min = np.sum((bsr - np.zeros_like(bsr)) ** 2) / bsr.size
    T = 25
    mse_av = 0.0
    for _ in range(T):
        population_rust = algorithm_rust.evolve(population_rust)
        bsr = population_rust.champion_x
        bvr = population_rust.champion_f
        mse = np.sum((bsr - (-2.903534 * np.ones_like(bsr))) ** 2) / bsr.size
        if (mse < mse_min):
            mse_min = mse
            bsr_best = bsr
            bvr_best = bvr
        elif (mse > mse_max):
            mse_max = mse
            bsr_worst = bsr
            bvr_worst = bvr



        mse_av += mse

    mse_av /=T
    info.append([mse_av, mse_min, mse_max, bvr_best[0], bvr_worst[0]])

In [9]:
for i in info:
    print(f'{i[0]} | {i[1]} | {i[2]} | {i[3]} | {i[4]}')

6.6391172812661905e-12 | 5.038341609657277e-15 | 3.2998316680032807e-11 | 2.000177801164682e-12 | 1.3093210071701833e-08
1.215781931600595e-10 | 1.2243081520664187e-13 | 1.899026155392632e-12 | 4.857980684391805e-11 | 7.535021495641558e-10
2.4318615611162555e-12 | 4.626081626992785e-16 | 1.4856954249121653e-11 | 1.8118839761882555e-13 | 5.895003596378956e-09
6.666890140833456e-12 | 2.9163989753206286e-14 | 1.3700597805057212e-11 | -78.33233140754147 | -78.33233140707118
2.8919085575611647e-11 | 2.8085769249462795e-15 | 2.4691815207638686e-12 | -78.33233140754268 | -78.33233140746012
4.1100593021451707e-11 | 1.0856331652890061e-14 | 2.001808196911347e-10 | -78.3323314075424 | -78.33233140062717
